In [1]:
%load_ext rpy2.ipython

In [49]:
import os
os.mkdir("./data")

In [2]:
%%R
library(dplyr)
library(ggplot2)
library(magrittr)

//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [13]:
%%R
#genome info files:

#kk preliminary groups
kk_phage_grps<-"/Users/jmb/Desktop/ViralFate/PhageGenomeAssembly/phage_by_group.txt"

#read coverage categories determined during phage genome assembly
read_coverage_categories<-"/Users/jmb/Desktop/ViralFate/ImportantInfo/read_coverage_patterns.txt"

#terminase categories based on closest BLAST hit
terminase_categories<-"/Users/jmb/Desktop/ViralFate/Terminase/data/new_terminase_categories.txt"
terminase_ortho_cluster<-"~/Desktop/ViralFate/Terminase/orthomcl/np_term_ortho_groups.txt"
terminase_ortho_defs<-"/Users/jmb/Desktop/ViralFate/Terminase/orthomcl/terminase_group_desc.txt"

#phage phylogeny determined by phylogeny of majority of POG annotations or else best BLAST match to POG
pog_phylogeny<-"/Users/jmb/Desktop/ViralFate/Annotation/nvp_pog_phy_simplified.txt"

#minimum similarity between hosts
host_sim<-"/Users/jmb/Desktop/ViralFate/OrthoMCL/phage_host_dist2.txt"

#nucleotide similarity: phages groups based on 50% genome similarity cutoff
nuc_divs<-"/Users/jmb/Desktop/ViralFate/Genome_Aln/data/nvp_nucl_comp_divs.txt"

#genome lengths
size<-"/Users/jmb/Desktop/ViralFate/ImportantInfo/igenome_lengths.txt"

#bc_hclust_grps
bhc<- "/Users/jmb/Desktop/NetworkFiles/data/new_nahantphage_clusters.txt"

     phage        cov_category
1 1.005.O. short_internal_peak
2 1.015.O. short_internal_peak
3 1.021.B. short_internal_peak
4 1.021.C. short_internal_peak
5 1.024.O. short_internal_peak
6 1.026.O. short_internal_peak


In [14]:
%%R
#consolidate terminase and coverage info:

term_blast_cat<-read.table(terminase_categories, sep="\t")
colnames(term_blast_cat)<-c("phage","eval","term_blast_ld", "term_blast_sd")
term_cluster<-read.table(terminase_ortho_cluster, sep="\t", header=TRUE)
term_clust_cats<-read.table(terminase_ortho_defs, sep="\t", header=TRUE)
#head(term_clust_cats)%>%print
#head(term_cluster)%>%print

term_cluster$term_ortho_sd<-term_clust_cats$short_desc[match(term_cluster$term_clust, term_clust_cats$terminase_grp)]
term_cluster$term_ortho_ld<-term_clust_cats$long_desc[match(term_cluster$term_clust, term_clust_cats$terminase_grp)]
nrow(term_blast_cat)%>%print
nrow(term_cluster)%>%print

read_cov<-read.table(read_coverage_categories, sep="\t", header=TRUE)

terminase_info<-merge(term_blast_cat, term_cluster, by="phage",all.x=TRUE, all.y=TRUE)

terminase_info<-merge(terminase_info, read_cov, by="phage", all.x=TRUE, all.y=TRUE)

nrow(terminase_info)%>%print

head(terminase_info)
#write.table()

[1] 238
[1] 263
[1] 283
     phage  eval term_blast_ld term_blast_sd  term_clust term_ortho_sd
1 1.003.O. 6e-63   headful/Sf6       Headful terminase16          <NA>
2 1.004.O. 2e-57    3-cos/HK97           Cos terminase23          <NA>
3 1.005.O. 1e-60    Cos/lambda           Cos  terminase3           Cos
4 1.007.O. 3e-65   headful/Sf6       Headful  terminase7          <NA>
5 1.009.O. 2e-57    Cos/lambda           Cos  terminase3           Cos
6 1.012.O. 2e-21   headful/Sf6       Headful  terminase1          <NA>
  term_ortho_ld            cov_category
1          <NA>           even_coverage
2          <NA> multiple_coverage_peaks
3    Cos/lambda     short_internal_peak
4          <NA> multiple_coverage_peaks
5    Cos/lambda                    <NA>
6          <NA>      shifts_in_coverage


In [15]:
%%R

phy<-read.table(pog_phylogeny, sep="\t", header=TRUE)
#phy%>%head%>%print
phy1<-phy[,c(1,2,4)]

host_min_sim<-read.table(host_sim, sep="\t", header=TRUE)
colnames(host_min_sim)[c(1,3)]<-c("phage","min_host_similarity")

genome_info<-host_min_sim[,c(1,2,3,6)]
colnames(genome_info)[4]<-"kk_grp"

nuc_grps<-read.table(nuc_divs, sep="\t", header=TRUE)
colnames(nuc_grps)[2]<-"nucleotide_level_divs"

sizzes<-read.table(size, sep="\t",header=TRUE)

genome_info<-merge(genome_info, phy1, by="phage", all.x=T, all.y=T)
genome_info<-merge(genome_info, nuc_grps, by="phage", all.x=T, all.y=T)
genome_info<-merge(genome_info, sizzes, by="phage", all.x=T, all.y=T)

head(genome_info)%>%print

     phage num_hosts min_host_similarity kk_grp        family          order
1 1.003.O.         2            76.77842      1  Podoviridae   Caudovirales 
2 1.004.O.         5            99.62443     14   Myoviridae   Caudovirales 
3 1.005.O.         1           100.00000      7  Podoviridae   Caudovirales 
4 1.007.O.         1           100.00000      1   Myoviridae   Caudovirales 
5 1.008.O.        36            73.57059     15    Tectivirus   Tectiviridae
6 1.009.O.         1           100.00000      7  Podoviridae   Caudovirales 
  nucleotide_level_divs illumina_genome_len
1                     1               41891
2                     2               42511
3                     3               50301
4                     4               49244
5                     5               10579
6                     6               44443


In [16]:
%%R
bray_clusters<-read.table("./data/bray_clusters_eps0.9_po1.txt", sep="\t",header=T)
euclidean_clusters<-read.table("./data/euc_clusters_eps15_po1.txt", sep="\t",header=T)

cluster_table<-merge(bray_clusters,euclidean_clusters, by="phage", all.x=T, all.y=T)
head(cluster_table)

     phage bray_clusters_eps0.9 euclidean_clusters_eps15
1 1.003.O.                    1                        1
2 1.004.O.                    2                        2
3 1.005.O.                    3                        3
4 1.007.O.                    4                        4
5 1.008.O.                    5                        5
6 1.009.O.                    0                        6


In [17]:
%%R
all<-merge(genome_info, cluster_table, by="phage", all.x=T, all.y=T)
all<-merge(all, terminase_info, by="phage", all.x=T, all.y=T)
head(all)

     phage num_hosts min_host_similarity kk_grp        family          order
1 1.003.O.         2            76.77842      1  Podoviridae   Caudovirales 
2 1.004.O.         5            99.62443     14   Myoviridae   Caudovirales 
3 1.005.O.         1           100.00000      7  Podoviridae   Caudovirales 
4 1.007.O.         1           100.00000      1   Myoviridae   Caudovirales 
5 1.008.O.        36            73.57059     15    Tectivirus   Tectiviridae
6 1.009.O.         1           100.00000      7  Podoviridae   Caudovirales 
  nucleotide_level_divs illumina_genome_len bray_clusters_eps0.9
1                     1               41891                    1
2                     2               42511                    2
3                     3               50301                    3
4                     4               49244                    4
5                     5               10579                    5
6                     6               44443                    0
  eucl

In [19]:
%%R
new_groups_sept<-read.table(bhc, sep="\t", header=TRUE)
colnames(new_groups_sept)<-c("phage","new_groups_sept")
all<-merge(all, new_groups_sept, by="phage", all.x=T, all.y=T)
head(all)

     phage num_hosts min_host_similarity kk_grp        family          order
1 1.003.O.         2            76.77842      1  Podoviridae   Caudovirales 
2 1.004.O.         5            99.62443     14   Myoviridae   Caudovirales 
3 1.005.O.         1           100.00000      7  Podoviridae   Caudovirales 
4 1.007.O.         1           100.00000      1   Myoviridae   Caudovirales 
5 1.008.O.        36            73.57059     15    Tectivirus   Tectiviridae
6 1.009.O.         1           100.00000      7  Podoviridae   Caudovirales 
  nucleotide_level_divs illumina_genome_len bray_clusters_eps0.9
1                     1               41891                    1
2                     2               42511                    2
3                     3               50301                    3
4                     4               49244                    4
5                     5               10579                    5
6                     6               44443                    0
  eucl

In [20]:
%%R
write.table(all, "./data/all_info.txt", sep="\t",row.names=F, quote=F)

In [57]:
%%R
max(na.omit(all$bray_clusters_eps0.9))

[1] 41
